In [1]:
# Import required libraries
import streamlit as st
from dotenv import load_dotenv

In [2]:
from PyPDF2 import PdfReader

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [7]:
from langchain.llms import HuggingFaceHub
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [9]:
!pip install htmltemplate

ImportError: cannot import name 'css' from 'htmltemplate' (/Users/pollipato/opt/anaconda3/envs/papita/lib/python3.9/site-packages/htmltemplate.py)

Extract text from PFD's

In [11]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

Split text into chunks

In [12]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

Create embedings and save on a vector store

In [14]:
def get_vectorstore(text_chunks, embedding_model):
    if embedding_model == "OpenAI":
        embeddings = OpenAIEmbeddings()
    elif embedding_model == "HuggingFace":
        embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    else:
        raise ValueError("Invalid language model choice. Supported options: 'OpenAI' and 'HuggingFace'.")

    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

Creating a conversational chain

In [15]:
def get_conversation_chain(vectorstore, conversational_model, model_temperature=0.5):
    if conversational_model == "OpenAI":
        llm = ChatOpenAI(temperature=model_temperature)
    elif conversational_model == "HuggingFace":
        llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory,
    )
    return conversation_chain

Handling user input

In [16]:
def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.write(user_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
        else:
            st.write(bot_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)

Building the Streamlit application

In [ ]:
def main():
    load_dotenv()
    st.set_page_config(page_title="Chat with multiple PDFs",
                       page_icon=":books:")
    st.write(css, unsafe_allow_html=True)

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = None

    st.header("Chat with multiple PDFs :books:")
    user_question = st.text_input("Ask a question about your documents:")

    # ... (omitted code for PDF upload and model selection)

    if user_question:
        handle_userinput(user_question)

if __name__ == '__main__':
    main()